<a href="https://colab.research.google.com/github/elliottbonal/Z3metaphysicstests/blob/main/Fourth_Theorem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install z3-solver
from z3 import *

# Defining the domain of all valid types (T143)
T143 = DeclareSort('T143')
x = Const('x', T143)

# Set membership predicates for \in Void and \in Portal
InVoid = Function('InVoid', T143, BoolSort())
InPortal= Function('InPortal', T143, BoolSort())

# Set-up for a reduced model of the Zmatrix being 5x5
N = 5
M = 5
Index = IntSort()

# Define predicates, Zmatrix, Z+ function and the zero_matrix
P = Function('P', Index, Index, T143, RealSort())  # Matrix entries
Z = Function('Z', T143, ArraySort(Index, ArraySort(Index, RealSort())))
Zplus = Function('Zplus', T143, RealSort())
zero_matrix = K(Index, K(Index, RealVal(0)))

# Setting up the theorem solver
s = Solver()

# Build matrix Z(x): Start with a Matrix full of zeros, and fill in each entry with predicate functions
Zx = K(Index, K(Index, RealVal(0)))
for i in range(1, N + 1):
    row = K(Index, RealVal(0))
    for j in range(1, M + 1):
        row = Store(row, j, P(i, j, x))
    Zx = Store(Zx, i, row)
s.add(Z(x) == Zx)

# Further define set membership as \in Void iff Z+ = 0 and \in Portal iff Z+(x) = 0 and Z+(x) > 0
s.add(ForAll([x], Implies(InVoid(x), Zplus(x) == 0)))
s.add(ForAll([x], Implies(InPortal(x), And(Zplus(x) == 0, Zplus(x) > 0))))

# Declaring that if an x is a member of the Void, then its Zmatrix is the zero matrix
s.add(ForAll([x], Implies(InVoid(x), Z(x) == zero_matrix)))

#Declaring that if the zero matrix is equal to itself, then the Zplus value of the x that it represents cannot be zero (as it satisfies at least P_1.1)
s.add(Implies(zero_matrix == zero_matrix, Zplus(x) > 0))

#Declaring that the zero matrix is equal to itself
s.add(zero_matrix == zero_matrix)

# Declaring that if Z+(x) = 0 and the zero matrix is equal to itself, then Z+(x) = 0 and Z+(x) > 0
s.add(ForAll([x], Implies(And(Zplus(x) == 0, zero_matrix == zero_matrix), And(Zplus(x) == 0, Zplus(x) > 0))))

# Trying to falsifiy the claim that The Portal is equal to the Void
s.push()
s.add(Not(ForAll([x], InPortal(x) == InVoid(x))))
result = s.check()

if result == unsat:
    print("No counterexample possible, the theorem holds.")
else:
    print("The theorem fails, counterexample found.")
    print("Model showing inconsistency:")
    print(s.model())



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 16.4 MB/s eta 0:00:00
No counterexample possible, the theorem holds.
